# Colab Imports

In [ ]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Installation

In [ ]:
!pip -q install langgraph langchain_openai langchain_community tiktoken konlpy

# Settings & Loadings

In [ ]:
import os
import json
import pandas as pd

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

PROFECT_DIR = os.path.join("/content/drive/MyDrive", "Profect SAE")

KRX_KOSPI = os.path.join(PROFECT_DIR, "KRX_KOSPI")
stock_embedding = os.path.join(PROFECT_DIR, "stocks_embedding")

# KOSPI

In [ ]:
## Kospi
krx_kospi = os.path.join(KRX_KOSPI, "encoded_company_no_bom.csv")
krx_kospi_categories = os.path.join(KRX_KOSPI, "companies_categories.csv")
kospi_abstract = os.path.join(stock_embedding, "kospi_900.csv")

krx_kospi_df = pd.read_csv(krx_kospi, encoding='utf-8-sig')
krx_kospi_categories_df = pd.read_csv(krx_kospi_categories, encoding='euc-kr')
kospi_abstract_df = pd.read_csv(kospi_abstract, encoding='utf-8-sig')

In [ ]:
kospi_abstract_df = kospi_abstract_df[kospi_abstract_df['company_abstract'] != 'Not Available']
krx_kospi_categories_df = krx_kospi_categories_df[krx_kospi_categories_df['종목코드'].isin(kospi_abstract_df['ticker_code'])]
krx_kospi_df = krx_kospi_df[krx_kospi_df['ISU_SRT_CD'].isin(kospi_abstract_df['ticker_code'])]

krx_kospi_categories_df = krx_kospi_categories_df.copy()
krx_kospi_categories_df.rename(columns={'종목코드': 'ticker_code'}, inplace=True)
krx_kospi_df = krx_kospi_df.copy()
krx_kospi_df.rename(columns={'ISU_SRT_CD': 'ticker_code'}, inplace=True)

combined_df = krx_kospi_df.merge(krx_kospi_categories_df, on='ticker_code', how='inner').merge(kospi_abstract_df, on='ticker_code', how='inner')

(819, 12)
(819, 8)
(819, 4)
(819, 22)


In [ ]:
combined_df_one = combined_df.loc[:, ['ISU_CD', 'ticker_code', 'ISU_NM', 'ISU_ABBRV', 'ISU_ENG_NM', 'MKT_TP_NM', '업종명', 'ur', 'company_abstract']].copy()
print(combined_df_one.columns)
combined_df_one.rename(columns={'ISU_CD': 'ticker_code_full'}, inplace=True)
combined_df_one.rename(columns={'ISU_NM': 'full_name'}, inplace=True)
combined_df_one.rename(columns={'ISU_ABBRV': 'korean_name'}, inplace=True)
combined_df_one.rename(columns={'ISU_ENG_NM': 'english_name'}, inplace=True)
combined_df_one.rename(columns={'MKT_TP_NM': 'market'}, inplace=True)
combined_df_one.rename(columns={'업종명': 'industry'}, inplace=True)
combined_df_one.rename(columns={'ur': 'url'}, inplace=True)
combined_df_one['country'] = "KOR"
print(combined_df_one.columns)
print(combined_df_one.shape)

combined_df_one.to_csv(os.path.join(stock_embedding, "before_embedding/kospi_900_info_v1.csv"), index=False)

Index(['ISU_CD', 'ticker_code', 'ISU_NM', 'ISU_ABBRV', 'ISU_ENG_NM',
       'MKT_TP_NM', '업종명', 'ur', 'company_abstract'],
      dtype='object')
Index(['ticker_code_full', 'ticker_code', 'full_name', 'korean_name',
       'english_name', 'market', 'industry', 'url', 'company_abstract',
       'country'],
      dtype='object')
(819, 10)


# Nasdaq

In [ ]:
## Nasdaq
nasdaq_abstract = os.path.join(stock_embedding, "nasdaq_1000.csv")
nasdaq_abstract_df = pd.read_csv(nasdaq_abstract, encoding='utf-8-sig')

nasdaq_abstract_df = nasdaq_abstract_df[nasdaq_abstract_df['company_abstract'] != 'Not Available']

# drop
nasdaq_abstract_df = nasdaq_abstract_df.drop(columns=['Unnamed: 0', 'url_abs'])

# columns
nasdaq_abstract_df['market'] = "NASDAQ"
nasdaq_abstract_df['ticker_code_full'] = "Nasdaq_"+nasdaq_abstract_df['ticker_code']
nasdaq_abstract_df['country'] = "USA"

# rename
nasdaq_abstract_df.rename(columns={'naver_stock_ticker': 'full_name'}, inplace=True)
nasdaq_abstract_df.rename(columns={'url_total': 'url'}, inplace=True)

# checking
nasdaq_abstract_df_columns = set(nasdaq_abstract_df.columns)
combined_df_one_columns = set(combined_df_one.columns)

common = combined_df_one_columns.intersection(nasdaq_abstract_df_columns)
nasdaq_abstract_df.to_csv(os.path.join(stock_embedding, "before_embedding/nasdaq_1000_info_v1.csv"), index=False)

(1000, 9)
Index(['Unnamed: 0', 'ticker_code', 'naver_stock_ticker', 'url_abs',
       'url_total', 'company_abstract', 'korean_name', 'english_name',
       'industry'],
      dtype='object')
Index(['ticker_code_full', 'ticker_code', 'full_name', 'korean_name',
       'english_name', 'market', 'industry', 'url', 'company_abstract',
       'country'],
      dtype='object')
Index(['ticker_code', 'full_name', 'url', 'company_abstract', 'korean_name',
       'english_name', 'industry', 'market', 'ticker_code_full', 'country'],
      dtype='object')
(819, 10)
(997, 10)
10 {'company_abstract', 'ticker_code', 'market', 'full_name', 'korean_name', 'english_name', 'ticker_code_full', 'country', 'industry', 'url'}
set()


# Industry: unique

In [ ]:
nasdaq_abstract_df = pd.read_csv(os.path.join(stock_embedding, "before_embedding/nasdaq_1000_info_v1.csv"), encoding='utf-8-sig')
kospi_abstract_df = pd.read_csv(os.path.join(stock_embedding, "before_embedding/kospi_900_info_v1.csv"), encoding='utf-8-sig')

print(nasdaq_abstract_df.shape)
print(kospi_abstract_df.shape)

kospi_industry = kospi_abstract_df['industry'].unique()
nasdaq_industry = nasdaq_abstract_df['industry'].unique()

(997, 10)
(819, 10)


In [ ]:
def map_industry_to_category(industry):
    # Convert to string and strip whitespace
    industry = str(industry).strip()

    # Financial Services (금융 서비스)
    if any(kw in industry for kw in ['금융', '은행', '증권', '보험', '핀테크', '투자', '중개', '대출', 'REITs']):
        return '금융 서비스'

    # Technology (기술)
    elif any(kw in industry for kw in ['소프트웨어', '반도체', 'IT', '하드웨어', '블록체인', '암호화폐', '네트워킹']):
        return '기술'

    # Healthcare (헬스케어)
    elif any(kw in industry for kw in ['제약', '의료', '생명 공학', '정밀기기', '헬스케어']):
        return '헬스케어'

    # Consumer Goods (소비재)
    elif any(kw in industry for kw in ['음식료', '담배', '의류', '음료', '가정용', '소비재', '장난감', '제화']):
        return '소비재'

    # Retail & Distribution (유통 및 소매)
    elif any(kw in industry for kw in ['유통', '소매', '백화점', '할인점', '온라인', '식품 소매']):
        return '유통 및 소매'

    # Manufacturing (제조업)
    elif any(kw in industry for kw in ['전기', '전자', '화학', '금속', '기계', '장비', '비금속', '종이', '목재', '항공우주']):
        return '제조업'

    # Transportation (운송)
    elif any(kw in industry for kw in ['운송', '창고', '물류', '항공사', '자동차', '트럭', '부품']):
        return '운송'

    # Utilities (유틸리티)
    elif any(kw in industry for kw in ['전기', '가스', '유틸리티', '재생 가능', '수자원']):
        return '유틸리티'

    # Real Estate (부동산)
    elif '부동산' in industry:
        return '부동산'

    # Entertainment & Leisure (엔터테인먼트 및 레저)
    elif any(kw in industry for kw in ['오락', '문화', '레스토랑', '호텔', '카지노', '방송', '엔터테인먼트']):
        return '엔터테인먼트 및 레저'

    # Construction & Materials (건설 및 자재)
    elif any(kw in industry for kw in ['건설', '엔지니어링', '자재', '주택']):
        return '건설 및 자재'

    # Telecommunications (통신)
    elif '통신' in industry:
        return '통신'

    # Energy (에너지)
    elif any(kw in industry for kw in ['오일', '가스', '석탄', '에너지']):
        return '에너지'

    # Agriculture & Natural Resources (농업 및 천연자원)
    elif any(kw in industry for kw in ['농업', '임업', '어업', '금', '채굴']):
        return '농업 및 천연자원'

    # Other Services (기타 서비스)
    else:
        return '기타 서비스'

In [ ]:
# Apply the mapping
kospi_abstract_df['industry_standardized'] = kospi_abstract_df['industry'].apply(map_industry_to_category)
nasdaq_abstract_df['industry_standardized'] = nasdaq_abstract_df['industry'].apply(map_industry_to_category)

In [ ]:
print(kospi_abstract_df.shape)
print(nasdaq_abstract_df.shape)

kospi_abstract_df.to_csv(os.path.join(stock_embedding, "before_embedding/kospi_900_info_v2.csv"), index=False)
nasdaq_abstract_df.to_csv(os.path.join(stock_embedding, "before_embedding/nasdaq_1000_info_v2.csv"), index=False)

(819, 11)
(997, 11)


# Vector Stores

## Data

In [ ]:
import pandas as pd

kospi_900_info = os.path.join(stock_embedding, "before_embedding/kospi_900_info_v2.csv")
nasdaq_1000_info = os.path.join(stock_embedding, "before_embedding/nasdaq_1000_info_v2.csv")

kospi_900_info_df = pd.read_csv(kospi_900_info, encoding='utf-8-sig')
nasdaq_1000_info_df = pd.read_csv(nasdaq_1000_info, encoding='utf-8-sig')

print(kospi_900_info_df.shape)
print(nasdaq_1000_info_df.shape)

(819, 11)
(997, 11)


## Secret Key

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Vector Init

### Installation

In [ ]:
!pip install -qU langchain-pinecone pinecone-notebooks

### Index and Embedding Models

In [ ]:
import time

from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=userdata.get('PINECONE-API-KEY'))

index_name = "sae-vectors"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

### Add Items

In [ ]:
from langchain_core.documents import Document

def document_generator(row):
  text = f"{row.english_name} {row.korean_name} {row.company_abstract} {row.industry}"
  text = text.lower().strip()
  return Document(
      page_content=text,
      metadata={
          'ticker_code_full': row.ticker_code_full,
          'ticker_code': row.ticker_code,
          'full_name': row.full_name,
          'korean_name': row.korean_name,
          'english_name': row.english_name,
          'market': row.market,
          'industry': row.industry,
          'url': row.url,
          'country': row.country,
          'industry_standardized': row.industry_standardized
      },
  )

In [ ]:
print(kospi_900_info_df.shape)
print(nasdaq_1000_info_df.shape)
kospi_900_info_df = kospi_900_info_df.dropna()
nasdaq_1000_info_df = nasdaq_1000_info_df.dropna()
print(kospi_900_info_df.shape)
print(nasdaq_1000_info_df.shape)
kospi_900_docs = kospi_900_info_df.apply(document_generator, axis=1)
nasdaq_1000_docs = nasdaq_1000_info_df.apply(document_generator, axis=1)

(819, 11)
(997, 11)
(819, 11)
(995, 11)


In [ ]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
count = 0
print("KOSPI 900")
for doc in kospi_900_docs:
  count+= len(encoding.encode(doc.page_content))
print("Total Token", count)
print("Avg Token: ", count/len(kospi_900_docs))

In [ ]:
from uuid import uuid4

# kospi 900
uuids = [str(uuid4()) for _ in range(len(kospi_900_docs))]
vector_store.add_documents(documents=kospi_900_docs, ids=uuids, namespace="kospi-900")

In [ ]:
from uuid import uuid4

# nasdaq 1000
uuids = [str(uuid4()) for _ in range(len(nasdaq_1000_docs))]
vector_store.add_documents(documents=nasdaq_1000_docs, ids=uuids, namespace="nasdaq-1000")